In [1]:
#Import Dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import pandas as pd

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

#### NASA Mars News

In [3]:
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html
soup = bs(html, "html.parser")

In [4]:
# Scrape latest news title and paragraph text using BeautifulSoup
news_title = soup.find("div",class_="content_title").text
news_paragraph = soup.find("div", class_="article_teaser_body").text

print(f"Title: {news_title}")
print(f"Text: {news_paragraph}")

Title: NASA's InSight Passes Halfway to Mars, Instruments Check In
Text: NASA's InSight spacecraft, en route to a Nov. 26 landing on Mars, passed the halfway mark on Aug. 6. All of its instruments have been tested and are working well.


#### JPL Mars Space Images - Featured Image

In [5]:
img_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_url)
html = browser.html
soup = bs(html, "lxml")

In [6]:
# Scrape featured image url using BeautifulSoup
img_url = soup.find("img", class_="thumb")["src"]
featured_img_url = "https://www.jpl.nasa.gov" + img_url
print(featured_img_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA22647-640x350.jpg


#### Mars Weather

In [7]:
url_weather = "https://twitter.com/marswxreport?lang=en"
browser.visit(url_weather)

In [8]:
# Scrape latest mars weather tweet using beautiful soup
html_weather = browser.html 
#soup = bs(html_weather, "html.parser")
soup = bs(html_weather, "lxml")
mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

print(mars_weather)

Sol 2145 (2018-08-19), high -10C/14F, low -66C/-86F, pressure at 8.68 hPa, daylight 05:29-17:43


#### Mars Facts

In [9]:
url_facts = "https://space-facts.com/mars/"
browser.visit(url_facts)

In [10]:
# Scrape profile table for Mars facts using Pandas
mars_facts = pd.read_html(url_facts)
mars_facts[0]

# Convert results to a dataframe
# Reset index
df_mars_facts = mars_facts[0]
df_mars_facts.columns = ['Parameter', 'Value']
df_mars_facts.set_index('Parameter', inplace = True)

df_mars_facts

,Value
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [11]:
# Generate HTML table from dataframe
mars_html_table = df_mars_facts.to_html()
mars_html_table = mars_html_table.replace('\n', '')
mars_html_table

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Value</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [12]:
# Save HTML table 
df_mars_facts.to_html('mars_facts_table.html')

#### Mars Hemispheres

In [13]:
import time 
url_hemispheres = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemispheres)
html = browser.html
soup = bs(html, "html.parser")

In [14]:
# Create a container to hold list of images
url_hemisphere_images = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

# Use splinter to loop through images
# load title and urls into a dictionary
for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + link    
    browser.visit(image_link)
    html = browser.html
    soup= bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    url_hemisphere_images.append({"title": title, "img_url": image_url})

In [15]:
url_hemisphere_images

[{'title': 'Cerberus Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere ',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]